<a href="https://colab.research.google.com/github/componavt/LLLE-R1900s/blob/main/src/visualization/radial_loan_stars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌐 Radial Loan Profiles: Bilingual Radar Charts per Settlement

Each village is a star — rays = credit categories.  
**Ray length** = total money borrowed.  
**Label at tip** = number of loans.

✨ **Features:**
- 🌍 Choose language: Russian (`ru`) or English (`en`)
- 🏘️ Settlement name: `Russian` or `English` from `society_settlement.csv`
- 📏 Axis labels: `loan_short_ru` (RU) or `Name` (EN) from `credit_items.csv`
- 🔢 Loan count shown as number
- 🖼️ Rendered directly in Colab

⚙️ Configurable via `USE_LANGUAGE` variable  
✅ Fully bilingual visualization

In [7]:
# Install compatible versions for static image rendering in Colab
#!pip install -q python-dotenv pandas plotly>=6.1.1 kaleido
!pip install -q python-dotenv pandas plotly

# Clone the repo if running in Colab (optional — if data not uploaded manually)
import os

if not os.path.exists('LLLE-R1900s'):
    !git clone https://github.com/componavt/LLLE-R1900s.git
    %cd LLLE-R1900s
else:
    %cd LLLE-R1900s

# === LANGUAGE SWITCH ===
USE_LANGUAGE = "ru"
assert USE_LANGUAGE in ("ru", "en"), "USE_LANGUAGE must be 'ru' or 'en'"

Cloning into 'LLLE-R1900s'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 162 (delta 86), reused 101 (delta 46), pack-reused 0 (from 0)
Receiving objects: 100% (162/162), 611.79 KiB | 4.13 MiB/s, done.
Resolving deltas: 100% (86/86), done.
/content/LLLE-R1900s/LLLE-R1900s/LLLE-R1900s


In [8]:
import os
import pandas as pd
from dotenv import load_dotenv

# Load configuration
load_dotenv('config.env')

# Paths
csv_out_dir = os.getenv('CSV_OUT_DIR', 'data/csv_out')
output_file_name = os.getenv('OUTPUT_CSV_FILE')

if not output_file_name:
    csv_files = [f for f in os.listdir(csv_out_dir) if f.endswith('.csv')]
    if not csv_files:
        raise FileNotFoundError("No CSV files found in the output directory.")
    output_file_name = csv_files[0]

csv_path = os.path.join(csv_out_dir, output_file_name)
print(f"Loading loan data from: {csv_path}")

# Load main loan data
df_loans = pd.read_csv(csv_path)
print(f"Loaded {len(df_loans)} loan records.")

# Load credit items (for axis labels)
df_credit = pd.read_csv('data/credit_items.csv')
print(f"Loaded {len(df_credit)} credit item definitions.")

# Load settlement metadata (for display names)
df_society = pd.read_csv('data/society_settlement.csv')
print(f"Loaded {len(df_society)} settlements.")

# --- Build settlement display name map ---
# Link: df_loans['settlement'] == df_society['English']
if USE_LANGUAGE == "ru":
    df_society['display_settlement'] = df_society['Russian']
else:
    df_society['display_settlement'] = df_society['English']

settlement_name_map = dict(zip(df_society['English'], df_society['display_settlement']))

# Validate settlements
unknown_settlements = set(df_loans['settlement']) - set(df_society['English'])
if unknown_settlements:
    print("⚠️ Warning: Unknown settlements in loan data:", unknown_settlements)

# --- Build credit item display label map ---
if USE_LANGUAGE == "ru":
    df_credit['display_label'] = df_credit['loan_short_ru']
else:
    df_credit['display_label'] = df_credit['Name']

label_map = dict(zip(df_credit['Name'], df_credit['display_label']))

# Validate credit items
unknown_items = set(df_loans['credit_item']) - set(df_credit['Name'])
if unknown_items:
    print("⚠️ Warning: Unknown credit_item values:", unknown_items)

# Add display settlement name to loan data
df_loans['display_settlement'] = df_loans['settlement'].map(settlement_name_map)
# Drop loans with unknown settlements (optional, but safe)
df_loans = df_loans.dropna(subset=['display_settlement'])

# Aggregate by display settlement and credit_item
df_agg = df_loans.groupby(['display_settlement', 'credit_item']).agg(
    total_amount=('amount_rubles', 'sum'),
    total_count=('loan_count', 'sum')
).reset_index()

print("✅ Data aggregated with bilingual labels.")

Loading loan data from: data/csv_out/loans_s4_i16.csv
Loaded 148 loan records.
Loaded 19 credit item definitions.
Loaded 29 settlements.
✅ Data aggregated with bilingual labels.


In [9]:
import plotly.graph_objects as go

# Consistent order of credit items
all_credit_items = sorted(df_credit['Name'].unique())
axis_labels = [label_map[item] for item in all_credit_items]

# Unique display settlement names
settlements = df_agg['display_settlement'].unique()
shown_count = 0

for settlement in settlements:
    df_set = df_agg[df_agg['display_settlement'] == settlement]

    # Build full vectors (including zeros)
    amounts = []
    counts = []
    for item in all_credit_items:
        match = df_set[df_set['credit_item'] == item]
        if not match.empty:
            amt = match['total_amount'].iloc[0]
            cnt = int(match['total_count'].iloc[0])
        else:
            amt = 0
            cnt = 0
        amounts.append(amt)
        counts.append(cnt)

    if sum(amounts) == 0:
        continue

    # Close the loop
    theta = axis_labels + [axis_labels[0]]
    r = amounts + [amounts[0]]
    text_labels = [str(c) if c > 0 else '' for c in counts] + ['']

    # Create figure
    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=r,
        theta=theta,
        fill='toself',
        line_color='steelblue',
        opacity=0.6,
        showlegend=False
    ))

    fig.add_trace(go.Scatterpolar(
        r=r,
        theta=theta,
        mode='markers+text',
        text=text_labels,
        textposition='top center',
        marker=dict(size=8, color='darkred'),
        showlegend=False
    ))

    title = f"Loan Profile: {settlement}" if USE_LANGUAGE == "en" else f"Профиль ссуд: {settlement}"
    radial_title = "Total Amount (Rubles)" if USE_LANGUAGE == "en" else "Сумма ссуд (руб.)"

    fig.update_layout(
        title=title,
        polar=dict(
            radialaxis=dict(visible=True, title=radial_title),
            angularaxis=dict(direction="clockwise")
        ),
        showlegend=False,
        width=650,
        height=650,
        font=dict(size=11)
    )

    fig.show(renderer="colab")
    shown_count += 1

print(f"\n🎉 Displayed {shown_count} radar charts in {'Russian' if USE_LANGUAGE == 'ru' else 'English'}.")


🎉 Displayed 4 radar charts in Russian.
